In [ ]:
import pandas as pd
import numpy as np
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, KFold
import lightgbm as lgb
import os
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import fbeta_score
import numpy as np

In [ ]:
train = pd.read_csv('/data/train_base.csv')
test = pd.read_csv('/data/testa_base.csv')
train_fea = pd.read_csv('train_trx_fea.csv')
test_fea = pd.read_csv('test_trx_fea.csv')

In [ ]:
train_fea_view = pd.read_csv('train_view_fea.csv')
test_fea_view = pd.read_csv('test_view_fea.csv')
view_emb = pd.read_csv('all_view_emb.csv')
w2v_view_emb = pd.read_csv('all_view_w2v_emb.csv')

In [ ]:
train = pd.merge(train, train_fea, on=['cust_wid'],how = 'left')
test = pd.merge(test, test_fea, on=['cust_wid'],how = 'left')

In [ ]:
train = pd.merge(train, train_fea_view, on=['cust_wid'],how = 'left')
test = pd.merge(test, test_fea_view, on=['cust_wid'],how = 'left')

In [ ]:
train = train.dropna(subset = ['age','gdr_cd'])
print(train.shape, test.shape)
train['label'][train.label>=1]=1

In [ ]:
cat_cols = ['gdr_cd','cty_cd']
question_cols = ['AACE49',
 'B571EC',
 '7501E5',
 '7940AB',
 '7E7E69',
 '8597A6',
 '8F1D43',
 '996009',
 '9C3B18',
 'A0872C',
 'A58149',
 'FB2FCD',
 'AF21D0',
 'B59C67',
 'F93882',
 'B9D487',
 'C6036A',
 'C60D06',
 'C667D5',
 'C6BFF6',
 'C9F95A',
 'D282EF',
 'D6EF5F',
 'DD7727',
 'E19347',
 'E58CC5',
 '5CAF41',
 '535AB7',
 '4E4E53',
 '20D135',
 '4DA040',
 'FD5C90',
 '0EEC27',
 '144A3F',
 '147702',
 'F7CADE',
 '2CBCA4',
 '3A15C7',
 '4476B9',
 '491442',
 '2DF452',
 'active_time_days',]
continue_cols = []
for col in train.columns:
    if col not in ['cust_wid','label']+cat_cols+question_cols:
        continue_cols.append(col)

for col in cat_cols:
    if col in train.columns:
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))  
train.head()

# 定义F2-score作为评价指标
def f2_score(y_true, y_pred):
    return 'f2', fbeta_score(np.round(y_true), np.array(y_pred.label), beta=2), True

# 训练函数
model_dir = os.path.abspath("models")
if not os.path.exists(model_dir): os.makedirs(model_dir)


In [ ]:
train.values[:,1].astype('int64')
datas = train[['cust_wid','label']+cat_cols+continue_cols]
label = 'label'

datas = shuffle(datas)
datas.reset_index(drop=True,inplace=True)

train, val = train_test_split(datas,test_size=0.2,random_state=67)
kf = KFold(n_splits=5,shuffle=True,random_state=7)    

_trainX,_trainY = train.values[:,2::],train.values[:,1].astype('int64')
valX,valY = val.values[:,2::],val.values[:,1].astype('int64')
valSet = lgb.Dataset(valX,valY,free_raw_data=False)  


params = {
'objective': 'multiclass',  
'num_class': 15,  
'metric': 'multi_logloss', #'objective': 'binary',
'boosting_type': 'gbdt',
'num_leaves': 31,
'learning_rate': 0.03,
'feature_fraction': 0.9,
'bagging_fraction': 0.8,
'bagging_freq': 1,
'verbose': -1,
'num_threads': 14
}


feat_columns=datas.columns.tolist()[2::]
print(feat_columns)

models = []
for idx,(tdx,vdx) in enumerate(kf.split(train)):
    trainX = _trainX[tdx]
    trainY = _trainY[tdx]
    _valX = _trainX[vdx]
    _valY = _trainY[vdx]
    trainSet = lgb.Dataset(trainX,trainY)
    _valSet = lgb.Dataset(_valX,_valY)
    model = lgb.train(params,trainSet,num_boost_round=40,valid_sets=[trainSet,_valSet,valSet],feature_name=feat_columns)#,feval=f2_score
    model.save_model("pre-a/models/model-{}.lgb".format(idx))
    models.append(model)


In [ ]:
pred = np.zeros(shape=(5,val.shape[0],15))

for idx,model in enumerate(models):
    pred[idx] = model.predict(valX)
print(pred.shape)
pred = pred.mean(axis=0)
print(pred.shape)
preds = np.argmax(pred, axis = 1)
preds

In [ ]:
def mape(y_true, y_pred):
    return 'mape', np.mean(np.abs((y_pred - y_true) / y_true)) * 100, True

sum(preds),sum(valY),mape(valY,np.array([2]*len(valY))),mape(valY,np.array([x if x>2 else 2 for x in preds ])),mape(valY,preds)

In [ ]:
def f2_metric(y_true, y_pred):
    return 'f2', fbeta_score(y_true, y_pred,average='binary',  beta=2), True

max_f2 = 0
max_id = 0
for i in np.arange(0,1,0.1):
    pred_binary = [1 if x >= i else 0 for x in pred]

    # 计算F2-score
    f2_score_val = f2_metric(valY, pred_binary)
    #print(f2_score_val)
    if max_f2<= f2_score_val[1]:
        max_f2 = f2_score_val[1]
        max_id = i

print("f2_score:",max_f2,'max_id:',max_id)

print(pred.shape,pred)
print(valY.shape,valY)
print(roc_auc_score(valY,pred))

In [ ]:
# 预测函数
def run_test(datas):
    datas = datas.values[:,1:]
    models = ["pre-a/models/model-{}.lgb".format(idx) for idx in range(5)]
    pred = np.zeros(shape=(5,datas.shape[0]))
    for idx,model_path in enumerate(models):
        model = lgb.Booster(model_file=model_path)
        pred[idx] = model.predict(datas)
    pred = pred.mean(axis=0)
    return pred

pred = run_test(test)
pred

res = pd.DataFrame(pred)

res = res.rename({0:'label'},axis = 1)
res.head()
print(len(res['label'][res.label>=0.2])/len(res))
res['label'][res.label>=0.2]=1
res['label'][res.label<0.2]=0
res['label'] = res['label'].astype(int)
out = pd.read_csv('data/testa_base.csv')

out['label'] = res['label'].astype(int)
out



out[['cust_wid','label']].sort_values('cust_wid').to_csv('output.csv',index = 0)

In [ ]:
out[['cust_wid','label']].sort_values('cust_wid').head()